In [1]:
import requests
import json

# Airtable credentials
AIRTABLE_BASE_ID = 'app7mcJMf9O8GQRal'
AIRTABLE_API_KEY = ''
TABLE_NAME = 'tiktok_post'
VIEW_NAME = '0_no_comment'
AIRTABLE_URL = f"https://api.airtable.com/v0/{AIRTABLE_BASE_ID}/{TABLE_NAME}"

# TikTok API credentials
TIKTOK_API_KEY = ""
TIKTOK_API_HOST = "tiktok-api23.p.rapidapi.com"
TIKTOK_API_URL = "https://tiktok-api23.p.rapidapi.com/api/post/comments"

# Headers for requests
airtable_headers = {
    "Authorization": f"Bearer {AIRTABLE_API_KEY}"
}

tiktok_headers = {
    "x-rapidapi-key": TIKTOK_API_KEY,
    "x-rapidapi-host": TIKTOK_API_HOST
}

# Lấy tất cả records từ Airtable
all_records = []
params = {"view": VIEW_NAME}

while True:
    response = requests.get(AIRTABLE_URL, headers=airtable_headers, params=params)
    response.raise_for_status()
    response_data = response.json()
    records = response_data.get("records", [])
    all_records.extend(records)
    
    offset = response_data.get("offset")
    if not offset:
        break
    params["offset"] = offset

def split_comments(formatted_comments, max_chars=100000):
    """
    Split comments list into smaller parts under max_chars,
    ensuring no comment is cut in the middle
    """
    parts = []
    current_part = []
    current_length = 0
    
    for comment in formatted_comments:
        # Add 2 for \n\n between comments
        comment_length = len(comment) + 2
        
        if current_length + comment_length > max_chars and current_part:
            # If current part is full, save it and create new part
            parts.append("\n\n".join(current_part))
            current_part = [comment]
            current_length = comment_length
        else:
            # Add comment to current part
            current_part.append(comment)
            current_length += comment_length
    
    # Add the last part if exists
    if current_part:
        parts.append("\n\n".join(current_part))
    
    return parts

# Process each record
for record in all_records:
    record_id = record['id']
    fields = record['fields']
    video_id = fields.get('video_id')

    if not video_id:
        # Update status for missing video_id
        update_data = {
            "fields": {
                "comments_status": "ERROR: Missing video_id"
            }
        }
        requests.patch(f"{AIRTABLE_URL}/{record_id}", headers=airtable_headers, json=update_data)
        print(f"Skipping record {record_id} - missing video_id")
        continue

    # Get comments for video
    cursor = 0
    all_comments = []
    formatted_comments = []
    
    try:
        while True:
            try:
                # Call API to get comments
                params = {
                    "videoId": video_id,
                    "count": 50,
                    "cursor": cursor
                }
                
                response = requests.get(TIKTOK_API_URL, headers=tiktok_headers, params=params)
                response.raise_for_status()
                data = response.json()
                
                # Save comments from response
                comments = data.get("comments", [])
                all_comments.extend(comments)
                
                # Format comments as requested
                for comment in comments:
                    user_id = comment.get('user', {}).get('unique_id', '')
                    text = comment.get('text', '')
                    formatted_comment = f"[{user_id}]\n[{text}]"
                    formatted_comments.append(formatted_comment)
                
                has_more = data.get("has_more")
                if not has_more:
                    break
                    
                cursor = data.get("cursor")
                
            except requests.exceptions.RequestException as e:
                error_msg = f"Error fetching comments for video {video_id}: {e}"
                update_data = {
                    "fields": {
                        "comments_status": f"ERROR: {error_msg}"
                    }
                }
                requests.patch(f"{AIRTABLE_URL}/{record_id}", headers=airtable_headers, json=update_data)
                print(error_msg)
                break

        # Update formatted comments to Airtable
        if formatted_comments:
            # Split comments into parts
            comment_parts = split_comments(formatted_comments)
            
            # Create update fields dictionary
            update_fields = {
                "comments_status": f"SUCCESS: {len(formatted_comments)} comments saved in {len(comment_parts)} parts"
            }
            
            # Add split comment parts
            for i, part in enumerate(comment_parts, 1):
                field_name = f"comments_{i}"
                update_fields[field_name] = part
            
            update_data = {
                "fields": update_fields
            }
            
            try:
                update_response = requests.patch(
                    f"{AIRTABLE_URL}/{record_id}", 
                    headers=airtable_headers, 
                    json=update_data
                )
                update_response.raise_for_status()
                
                # Print status with detailed information
                status_msg = (
                    f"Record {record_id} processed successfully:\n"
                    f"- Total comments: {len(formatted_comments)}\n"
                    f"- Split into {len(comment_parts)} parts:\n"
                )
                
                for i, part in enumerate(comment_parts, 1):
                    comment_count = part.count('[') // 2
                    char_count = len(part)
                    status_msg += f"  Part {i}: {comment_count} comments, {char_count} characters\n"
                    
                print(status_msg)
                
            except requests.exceptions.RequestException as e:
                error_msg = f"Error updating record {record_id}: {e}"
                update_data = {
                    "fields": {
                        "comments_status": f"ERROR: {error_msg}"
                    }
                }
                requests.patch(f"{AIRTABLE_URL}/{record_id}", headers=airtable_headers, json=update_data)
                print(error_msg)
        else:
            # No comments found
            update_data = {
                "fields": {
                    "comments_status": "NO_COMMENTS: No comments found for this video"
                }
            }
            requests.patch(f"{AIRTABLE_URL}/{record_id}", headers=airtable_headers, json=update_data)
            print(f"No comments found for record {record_id}")
            
    except Exception as e:
        # Catch any unexpected errors
        error_msg = f"Unexpected error processing record {record_id}: {e}"
        update_data = {
            "fields": {
                "comments_status": f"ERROR: {error_msg}"
            }
        }
        requests.patch(f"{AIRTABLE_URL}/{record_id}", headers=airtable_headers, json=update_data)
        print(error_msg)



Record reclQ2XnJFCDxZnCt processed successfully:
- Total comments: 982
- Split into 1 parts:
  Part 1: 982 comments, 41003 characters



KeyboardInterrupt: 